In [275]:
import pandas as pd
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
import matplotlib

In [277]:
dump=pd.read_csv("dump.csv")

In [278]:
sel_id=[2,3,5,9,10,11,12,13,14,15,16,17,18,20]

In [268]:
def f_cast(k):
    sel=sel=dump[dump.SellerId==k][["TransactionDate","Amount"]]
    sel["TransactionDate"]=sel.TransactionDate.apply(lambda x: x.split("T")[0])
    sel["TransactionDate"]=pd.to_datetime(sel.TransactionDate)
    sel = sel.groupby("TransactionDate").sum()
    ts=sel["Amount"]
    #print(ts.head())
    def dft(ts):
        print ('Results of Dickey-Fuller Test:')
        dftest = adfuller(ts, autolag='AIC')
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        for key,value in dftest[4].items():
            dfoutput['Critical Value (%s)'%key] = value
        return(dfoutput)
    #print(dft(ts))
    ts_log=np.log(ts[ts>0])
    ts_log_diff=ts_log-ts_log.shift()
    ts_log_diff.dropna(inplace=True)
    #print(dft(ts_log_diff))
    from statsmodels.tsa.stattools import acf, pacf
    lag_acf = acf(ts_log_diff, nlags=1)
    lag_pacf = pacf(ts_log_diff, nlags=1, method='ols')
    def acf(lag_acf):
        #Plot ACF: 
        plt.subplot(121) 
        acf=plt.plot(lag_acf)
        plt.axhline(y=0,linestyle='--',color='gray')
        plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
        plt.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
        plt.title('ACF')
        return acf
    #acf(lag_acf)
    def pacf(lag_pacf):
        #Plot PACF:
        plt.subplot(122)
        pacf=plt.plot(lag_pacf)
        plt.axhline(y=0,linestyle='--',color='gray')
        plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
        plt.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
        plt.title('PACF')
        plt.tight_layout()
        return pacf
    #pacf(lag_pacf)
    def fcast(ts_log_diff):
        mod = sm.tsa.statespace.SARIMAX(ts_log_diff,
                                        order=(1, 1, 1),
                                        seasonal_order=(1, 1, 1, 8),
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)

        results = mod.fit()

        pred_uc=results.get_forecast(steps=25)
        pred_ci = pred_uc.conf_int()
        pred_mean=(pred_ci.iloc[:,0]+pred_ci.iloc[:,1])/2
        predictions_ARIMA_diff = pd.Series(pred_mean, copy=True)
        predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
        a=ts_log[-1]
        b=a+predictions_ARIMA_diff_cumsum
        predictions_ARIMA = np.exp(b)
        return predictions_ARIMA
    output=fcast(ts_log_diff)
    return output

In [269]:
seller_wise_forecast=pd.DataFrame()
for k in sel_id:
    seller_wise_forecast["seller_id"+str(k)]=f_cast(k).reset_index(drop=True)   

In [270]:
lis=[1,2,6,7,8,19]
for k in lis:
    seller_wise_forecast["seller_id"+str(k)]=dump[dump.SellerId==k].Amount.mean()

In [271]:
seller_wise_forecast

,seller_id2,seller_id3,seller_id5,seller_id9,seller_id10,seller_id11,seller_id12,seller_id13,seller_id14,seller_id15,seller_id16,seller_id17,seller_id18,seller_id20,seller_id1,seller_id6,seller_id7,seller_id8,seller_id19
0,1.052295e+06,9.061047e+05,98626.095711,2.195431e+07,6.664294e+06,105530.658597,1.177192e+06,6.988194e+06,7.906039e+06,1.799644e+06,145066.497292,6.161455e+06,7.733735e+05,36191.314580,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
1,1.052295e+06,3.134263e+04,58862.999365,5.890452e+07,1.657151e+07,11705.339221,1.393143e+06,8.457656e+06,6.236401e+06,1.896983e+06,239512.879816,2.143633e+06,6.762287e+05,27702.603584,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
2,1.052295e+06,6.163858e+04,125475.510000,3.340703e+07,8.546371e+06,25818.247224,6.361306e+05,7.529321e+06,7.736853e+06,2.673559e+06,300972.731121,2.653703e+06,8.809846e+05,32777.442954,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
3,1.052295e+06,1.487266e+06,107975.811690,1.245430e+08,2.411651e+07,35731.350189,8.728759e+05,1.302162e+07,6.403057e+06,2.230368e+06,322919.293479,3.621699e+06,7.284730e+05,23025.545123,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
4,1.052295e+06,2.755149e+04,87707.558734,5.530107e+07,7.597075e+06,15719.503573,7.307966e+05,8.431782e+06,1.097512e+07,1.802207e+06,190354.024856,1.927848e+06,6.480501e+05,30714.191204,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
5,1.052295e+06,2.175134e+05,78074.237726,7.291880e+07,1.296119e+07,30271.425211,6.375351e+05,1.486468e+07,5.918486e+06,1.084835e+06,149541.431001,1.014755e+06,1.030501e+06,28084.644602,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
6,1.052295e+06,1.457630e+04,69962.990441,7.694884e+07,8.604969e+06,54693.367920,6.487135e+05,8.243533e+06,6.440077e+06,1.706600e+06,147026.240502,3.251801e+06,5.788848e+05,28291.464591,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
7,1.052295e+06,9.811263e+03,87467.866104,1.664179e+07,5.040895e+06,114031.681088,4.363377e+05,1.386796e+07,5.811488e+06,2.865816e+06,139971.023306,3.176331e+06,6.348904e+05,34059.146331,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
8,1.052295e+06,5.023483e+04,93320.525743,1.905804e+07,1.640383e+07,59588.446928,1.265696e+06,1.124719e+07,9.431856e+06,1.728342e+06,205679.455633,2.657980e+06,7.974653e+05,34558.113588,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
9,1.052295e+06,2.515239e+04,68201.418329,6.776626e+07,9.954421e+06,11778.049485,1.067334e+06,1.628977e+07,6.819789e+06,2.200418e+06,163334.153861,3.310526e+06,6.610983e+05,29282.962120,7900.0,3901446.875,NaN,2.299965e+06,2505000.0


In [272]:
seller_wise_forecast.to_csv("Seller_Wise_Forecast.csv",index=False)

In [273]:
df=pd.read_csv("Seller_Wise_Forecast.csv")

In [274]:
df.head()

,seller_id2,seller_id3,seller_id5,seller_id9,seller_id10,seller_id11,seller_id12,seller_id13,seller_id14,seller_id15,seller_id16,seller_id17,seller_id18,seller_id20,seller_id1,seller_id6,seller_id7,seller_id8,seller_id19
0,1.052295e+06,9.061047e+05,98626.095711,2.195431e+07,6.664294e+06,105530.658597,1.177192e+06,6.988194e+06,7.906039e+06,1.799644e+06,145066.497292,6.161455e+06,773373.482594,36191.314580,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
1,1.052295e+06,3.134263e+04,58862.999365,5.890452e+07,1.657151e+07,11705.339221,1.393143e+06,8.457656e+06,6.236401e+06,1.896983e+06,239512.879816,2.143633e+06,676228.690218,27702.603584,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
2,1.052295e+06,6.163858e+04,125475.510000,3.340703e+07,8.546371e+06,25818.247224,6.361306e+05,7.529321e+06,7.736853e+06,2.673559e+06,300972.731121,2.653703e+06,880984.635465,32777.442954,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
3,1.052295e+06,1.487266e+06,107975.811690,1.245430e+08,2.411651e+07,35731.350189,8.728759e+05,1.302162e+07,6.403057e+06,2.230368e+06,322919.293479,3.621699e+06,728473.035725,23025.545123,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
4,1.052295e+06,2.755149e+04,87707.558734,5.530107e+07,7.597075e+06,15719.503573,7.307966e+05,8.431782e+06,1.097512e+07,1.802207e+06,190354.024856,1.927848e+06,648050.086776,30714.191204,7900.0,3901446.875,NaN,2.299965e+06,2505000.0
